# Variational Multi-modal Recurrent Graph AutoEncoder
In this tuorial, we will go through how to run a Variational Multi-modal Recurrent Graph AutoEncoder (VMR-GAE) model for origin-destination (OD) matrix completion. In particular, we will demonstrate how to train the model and evaluate the completion results.

## Part I: Training
In this part, we will show how to train a VMR-GAE model for OD matrix completion on the NYC taxi dataset. In particular, we adopt some training skills from previous works, including data normalization, Kullback-Leibler loss delay and so on.

Visit `paddlespatial/networks/vmrgae/train.py` for more details.

In [1]:
import argparse
import os
import numpy as np
import paddle
import pgl
from model import VmrGAE
import utils as utils
from utils import MinMaxScaler

/opt/anaconda3/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:943: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  collections.MutableMapping.register(ParseResults)
/opt/anaconda3/lib/python3.8/site-packages/pkg_resources/_vendor/pyparsing.py:3226: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  elif isinstance( exprs, collections.Iterable ):


The VmrGAE class is built upon PaddlePaddle, which is a deep learning framework.

In [2]:
def prep_env(flag='train'):
    # type: (str) -> dict
    """
    Desc:
        Prepare the environment
    Args:
        flag: specify the environment, 'train' or 'evaluate'
    Returns:
        A dict indicating the environment variables
    """
    parser = \
        argparse.ArgumentParser(description='{} [VMR-GAE] on the task of OD Matrix Completion'
                                .format("Training" if flag == "train" else "Evaluating"))
    parser.add_argument('--num_nodes', type=int, default=263, help='The number of nodes in the graph')
    parser.add_argument('--timelen', type=int, default=3, help='The length of input sequence')
    parser.add_argument('--hidden_dim', type=int, default=32, help='The dimensionality of the hidden state')
    parser.add_argument('--rnn_layer', type=int, default=2, help='The number of RNN layers')
    parser.add_argument('--delay', type=int, default=0, help='delay to apply kld_loss')
    parser.add_argument('--clip_max_value', type=int, default=1, help='clip the max value')
    parser.add_argument('--align', type=bool, default=True,
                        help='Whether or not align the distributions of two modals')
    parser.add_argument('--x_feature', type=bool, default=False,
                        help='X is a feature matrix (if True) or an identity matrix (otherwise)')
    parser.add_argument('--data_path', type=str, default='./data/NYC-taxi', help='Data path')
    parser.add_argument('--checkpoints', type=str, default='./nyc/checkpoints', help='Checkpoints path')
    parser.add_argument('--device', type=str, default='cpu', help='cpu or gpu')
    if flag == "train":
        parser.add_argument('--iter_num', type=int, default=10, help='The number of iterations')
        parser.add_argument('--learning_rate', type=float, default=0.0001, help='delay to apply kld_loss')
        parser.add_argument('--result_path', type=str, default='./nyc/results', help='result path')
    else:
        parser.add_argument('--sample_time', type=int, default=10, help='The sample time for point estimation')

    args = parser.parse_known_args()[0]

    if flag == "train":
        if not os.path.exists(args.checkpoints):
            os.makedirs(args.checkpoints)
        if not os.path.exists(args.result_path):
            os.makedirs(args.result_path)
    else:
        if not os.path.exists(args.checkpoints):
            print('Checkpoint does not exist.')
            exit()

    primary_flow = np.load('%s/train_data.npy' % args.data_path, allow_pickle=True)
    supp_flow = np.load('%s/green_data.npy' % args.data_path, allow_pickle=True)
    train_data = np.load('%s/train_data.npy' % args.data_path, allow_pickle=True)[-1]
    val_data = np.load('%s/val_data.npy' % args.data_path, allow_pickle=True)
    test_data = np.load('%s/test_data.npy' % args.data_path, allow_pickle=True)

    # scaling data
    ground_truths = []
    for i in range(len(primary_flow)):
        primary_flow[i][0] = np.array(primary_flow[i][0]).astype("int")
        primary_flow[i][1] = np.array(primary_flow[i][1]).astype("float32")
        ground_truths.append(utils.index_to_adj_np(primary_flow[i][0], primary_flow[i][1], args.num_nodes))
    ground_truths = np.stack(ground_truths, axis=0)
    if args.clip_max_value == 1:
        max_value = 50
    else:
        print(np.concatenate(primary_flow[:, 1]).max())
        max_value = np.concatenate(primary_flow[:, 1]).max()
    primary_scale = MinMaxScaler(0, max_value)
    for i in range(args.timelen):
        primary_flow[i][1] = primary_scale.transform(primary_flow[i][1])

    for i in range(len(supp_flow)):
        supp_flow[i][0] = np.array(supp_flow[i][0]).astype("int")
        supp_flow[i][1] = np.array(supp_flow[i][1]).astype("float32")
    supp_scale = MinMaxScaler(0, np.concatenate(supp_flow[:, 1]).max())
    for i in range(args.timelen):
        supp_flow[i][1] = supp_scale.transform(supp_flow[i][1])

    # load into paddle
    mask = np.zeros((args.num_nodes, args.num_nodes))
    for i in range(args.timelen):
        mask[np.where(ground_truths[i] > (2 / max_value))] = 1.0

    target_graph = []
    for i in range(len(primary_flow)):
        target_graph.append(pgl.Graph(edges=primary_flow[i][0],
                                      num_nodes=args.num_nodes,
                                      edge_feat={'efeat': paddle.to_tensor(primary_flow[i][1])}))
    supp_graph = []
    for i in range(len(primary_flow)):
        supp_graph.append(pgl.Graph(edges=supp_flow[i][0],
                                    num_nodes=args.num_nodes,
                                    edge_feat={'efeat': paddle.to_tensor(supp_flow[i][1])}))

    mask = paddle.to_tensor(mask)
    xs = paddle.to_tensor([np.eye(args.num_nodes) for i in range(args.timelen)])
    x = paddle.to_tensor([np.eye(args.num_nodes) for i in range(args.timelen)])
    ground_truths = paddle.to_tensor(ground_truths, dtype='float32')

    res = {
        "args": args,
        "primary_flow": primary_flow, "primary_scale": primary_scale, "target_graph": target_graph, "x": x,
        "mask": mask,
        # "supp_flow": supp_flow, "supp_scale": supp_scale,
        "supp_graph": supp_graph, "xs": xs,
        "ground_truths": ground_truths,
        "train_data": train_data, "val_data": val_data, "test_data": test_data
    }
    return res

/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Environment Preparation
Here we use `argparse` method for arguments setting, including model parameters, file paths and arguments for training. Then, we load the data from the given path and scale them with normalization process. Since we use `PaddlePaddle` as backend, we should also transform the data into `paddle.tensor` form. Note that we use the iteration number as 10 only for demonstration and it need a larger number for training (e.g., 10e5).

In [3]:
if __name__ == '__main__':
    env = prep_env()
    if env['args'].device=='gpu':
        paddle.set_device('gpu')

min: 0 max: 50
min: 0 max: 25.0


### Load the model and settings
The class is defined in the `paddlespatial/networks/vmrgae/model.py`. 

Check it for more details.

In [4]:
    model = VmrGAE(x_dim=env["x"].shape[-1], d_dim=env["xs"].shape[-1], h_dim=env["args"].hidden_dim,
                   num_nodes=env["args"].num_nodes, n_layers=env["args"].rnn_layer,
                   eps=1e-10, same_structure=True)

Before training, read the checkpoints if available

In [5]:
    if not os.path.isfile('%s/model.pdparams' % env["args"].checkpoints):
        print("Start new train (model).")
        min_loss = np.Inf
        epoch = 0
    else:
        print("Found the model file. continue to train ... ")
        model.set_state_dict(paddle.load('%s/model.pdparams' % env["args"].checkpoints))
        min_loss = paddle.load('%s/minloss.pdtensor' % env["args"].checkpoints)
        epoch = np.load('%s/logged_epoch.npy' % env["args"].checkpoints)

Found the model file. continue to train ... 


In [6]:
    optimizer = paddle.optimizer.Adam(learning_rate=env["args"].learning_rate, parameters=model.parameters())
    if os.path.isfile('%s/opt_state.pdopt' % env["args"].checkpoints):
        opt_state = paddle.load('%s/opt_state.pdopt' % env["args"].checkpoints)
        optimizer.set_state_dict(opt_state)
    patience = np.Inf
    best_val_mape = np.Inf
    max_iter = 0

### Start train
We now initialize the Adam optimizer and start the training procedure. The learning rate is set to 0.0001. Here we can activate the early stop mechanism if need. Then we train the model for 10 epochs for demostration purposes. In each epoch, we receive the losses, the critical intermediate variables, and the completed OD matrix. If the loss goes down, we then save the checkpoint.

In [7]:
    for k in range(epoch, env["args"].iter_num):
        kld_loss_tvge, kld_loss_avde, pis_loss, all_h, all_enc_mean, all_prior_mean, all_enc_d_mean, all_dec_t, \
            all_z_in, all_z_out \
            = model(env["x"], env["xs"], env["target_graph"], env["supp_graph"], env["mask"],
                    env["primary_scale"], env["ground_truths"])
        pred = env["primary_scale"].inverse_transform(all_dec_t[-1].numpy())
        val_MAE, val_RMSE, val_MAPE = utils.validate(pred, env["val_data"][0],
                                                     env["val_data"][1], flag='val')
        test_MAE, test_RMSE, test_MAPE = utils.validate(pred, env["test_data"][0],
                                                        env["test_data"][1], flag='test')
        if val_MAPE < best_val_mape:
            best_val_mape = val_MAPE
            max_iter = 0
        else:
            max_iter += 1
            if max_iter >= patience:
                print('Early Stop!')
                break
        if k >= env["args"].delay:
            loss = kld_loss_tvge + kld_loss_avde + pis_loss
        else:
            loss = pis_loss
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        if k % 10 == 0:
            print('epoch: ', k)
            print('loss =', loss.mean().item())
            print('kld_loss_tvge =', kld_loss_tvge.mean().item())
            print('kld_loss_avde =', kld_loss_avde.mean().item())
            print('pis_loss =', pis_loss.mean().item())
            print('val', "MAE:", val_MAE, 'RMSE:', val_RMSE, 'MAPE:', val_MAPE)
            print('test', "MAE:", test_MAE, 'RMSE:', test_RMSE, 'MAPE:', test_MAPE)

        if (loss.mean() < min_loss).item() | (k == env["args"].delay):
            print('epoch: %d, Loss goes down, save the model. pis_loss = %f' % (k, pis_loss.mean().item()))
            print('val', "MAE:", val_MAE, 'RMSE:', val_RMSE, 'MAPE:', val_MAPE)
            print('test', "MAE:", test_MAE, 'RMSE:', test_RMSE, 'MAPE:', test_MAPE)
            min_loss = loss.mean().item()
            paddle.save(all_enc_mean, '%s/all_enc_mean.pdtensor' % env["args"].result_path)
            paddle.save(all_prior_mean, '%s/all_prior_mean.pdtensor' % env["args"].result_path)
            paddle.save(all_enc_d_mean, '%s/all_enc_d_mean.pdtensor' % env["args"].result_path)
            paddle.save(all_dec_t, '%s/all_dec_t.pdtensor' % env["args"].result_path)
            paddle.save(all_z_in, '%s/all_z_in.pdtensor' % env["args"].result_path)
            paddle.save(all_z_out, '%s/all_z_out.pdtensor' % env["args"].result_path)
            paddle.save(model.state_dict(), '%s/model.pdparams' % env["args"].checkpoints)
            paddle.save(loss.mean(), '%s/minloss.pdtensor' % env["args"].checkpoints)
            paddle.save(optimizer.state_dict(), '%s/opt_state.pdopt' % env["args"].checkpoints)
            np.save('%s/logged_epoch.npy' % env["args"].checkpoints, k)

/opt/anaconda3/lib/python3.8/site-packages/paddle/fluid/dygraph/math_op_patch.py:251: UserWarning: The dtype of left and right variables are not the same, left dtype is paddle.float32, but right dtype is paddle.float64, the right dtype will convert to paddle.float32
  warnings.warn(


epoch: 2, Loss goes down, save the model. pis_loss = 44.056442
val MAE: 16.036873800104313 RMSE: 17.80933483450297 MAPE: 2.8293304056819606
test MAE: 16.345251850053376 RMSE: 21.365916076839955 MAPE: 2.3583845337423264
epoch: 3, Loss goes down, save the model. pis_loss = 42.328651
val MAE: 15.313282637162642 RMSE: 17.06790515321285 MAPE: 2.70043636217129
test MAE: 15.499441642387241 RMSE: 20.93153726360367 MAPE: 2.1718683112669486
epoch: 4, Loss goes down, save the model. pis_loss = 41.584705
val MAE: 14.28275656266646 RMSE: 15.98162930406986 MAPE: 2.472713580716669
test MAE: 15.514242705176859 RMSE: 21.007513429655813 MAPE: 2.2006020150986467
epoch: 5, Loss goes down, save the model. pis_loss = 38.909363
val MAE: 14.249638609452681 RMSE: 15.946169632340105 MAPE: 2.430129598982639
test MAE: 14.807489628885307 RMSE: 20.77644899688127 MAPE: 2.074672180305502
epoch: 6, Loss goes down, save the model. pis_loss = 38.290962
val MAE: 13.810019891912287 RMSE: 15.620660113793411 MAPE: 2.4010547

The above is about the training steps, you can adjust it as needed.

## Part II: Result Evalution
Below we will introduce how to use the trained model for OD matrix completion and evaluate the results.

Visit `paddlespatial/networks/vmrgae/eval.py` for more details.

In [9]:
from train import prep_env
if __name__ == '__main__':
    env = prep_env(flag='eval')
    if env['args'].device=='gpu':
        paddle.set_device('gpu')

    # load VMR-GAE and run
    model = VmrGAE(x_dim=env["x"].shape[-1], d_dim=env["xs"].shape[-1], h_dim=env["args"].hidden_dim,
                   num_nodes=env["args"].num_nodes, n_layers=env["args"].rnn_layer,
                   eps=1e-10, same_structure=True)

    if not os.path.isfile('%s/model.pdparams' % env["args"].checkpoints):
        print('Checkpoint does not exist.')
        exit()
    else:
        model.set_state_dict(paddle.load('%s/model.pdparams' % env["args"].checkpoints))
        min_loss = paddle.load('%s/minloss.pdtensor' % env["args"].checkpoints)
        epoch = np.load('%s/logged_epoch.npy' % env["args"].checkpoints)

min: 0 max: 50
min: 0 max: 25.0


Here we use the same preparation function in training process with `eval` flag to hold the model configurations. Then, we load the model and the available checkpoint.

### Start Evaluation
We perform the trained model for `sample_time` times and report the mean values as the completion results, as well as the standard deviations.

In [10]:
    pred = []
    for i in range(env["args"].sample_time):
        _, _, _, _, _, _, _, all_dec_t, _, _ \
            = model(env["x"], env["xs"], env["target_graph"], env["supp_graph"], env["mask"],
                    env["primary_scale"], env["ground_truths"])
        pred.append(env["primary_scale"].inverse_transform(all_dec_t[-1].numpy()))
    pred = np.stack(pred, axis=0)
    pe, std = pred.mean(axis=0), pred.std(axis=0)
    pe[np.where(pe < 0.5)] = 0
    print(pe)

[[20.457312 22.099165 22.369968 ... 20.148964 20.399767 19.612144]
 [19.998463 21.68787  21.64738  ... 19.538559 20.009933 19.260576]
 [20.739218 22.463572 21.908607 ... 20.383247 20.573793 20.101337]
 ...
 [20.076315 21.49416  21.676641 ... 19.469728 19.679989 18.836117]
 [19.822569 21.379684 20.878109 ... 19.325747 19.726997 19.063314]
 [19.820883 21.541578 21.235636 ... 19.330482 19.592773 19.105854]]
